In [1]:
import pandas as pd
import datetime
import numpy as np

In [2]:
df= pd.read_csv('df_nbh.csv')

In [3]:
df.head()

,incident_datetime,neighborhood,day_of_week,hour_of_day,incident_id
0,2011-06-07 00:00:00,NaN,Tuesday,12,14
1,2011-05-28 00:00:00,NaN,Saturday,12,12
2,2011-06-03 00:00:00,NaN,Friday,12,12
3,2011-06-12 00:00:00,NaN,Sunday,12,12
4,2011-05-30 00:00:00,NaN,Monday,11,11


In [4]:
df.columns=['date', 'nbh', 'day', 'hour', 'nb']
df["day"] = df["day"].replace(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY"])

In [5]:
df['nbh'].unique()

array([nan, 'North Park', 'Lovejoy', 'Pratt-Willert', 'Riverside',
       'Broadway Fillmore', 'Elmwood Bryant', 'Central', 'West Hertel',
       'Allentown', 'Elmwood Bidwell', 'Genesee-Moselle', 'Kaisertown',
       'University Heights', 'Kensington-Bailey', 'West Side',
       'Masten Park', 'Upper West Side', 'Lower West Side', 'Black Rock',
       'Fruit Belt', 'Schiller Park', 'Fillmore-Leroy',
       'Seneca-Cazenovia', 'Kenfield', 'Parkside', 'Hopkins-Tifft',
       'Central Park', 'South Park', 'Ellicott', 'MLK Park',
       'Grant-Amherst', 'Delavan Grider', 'Hamlin Park', 'Seneca Babcock',
       'First Ward'], dtype=object)

In [6]:
df["nbh"] = df["nbh"].replace(['North Park', 'Lovejoy', 'Pratt-Willert', 'Riverside',
                                    'Broadway Fillmore', 'Elmwood Bryant', 'Central', 'West Hertel',
                                   'Allentown', 'Elmwood Bidwell', 'Genesee-Moselle', 'Kaisertown',
                                   'University Heights', 'Kensington-Bailey', 'West Side',
                                   'Masten Park', 'Upper West Side', 'Lower West Side', 'Black Rock',
                                   'Fruit Belt', 'Schiller Park', 'Fillmore-Leroy',
                                   'Seneca-Cazenovia', 'Kenfield', 'Parkside', 'Hopkins-Tifft',
                                   'Central Park', 'South Park', 'Ellicott', 'MLK Park',
                                   'Grant-Amherst', 'Delavan Grider', 'Hamlin Park', 'Seneca Babcock',
                                   'First Ward' ], ['NorthPark', 'Lovejoy', 'PrattWillert', 'Riverside',
                                                    'BroadwayFillmore', 'ElmwoodBryant', 'Central', 'WestHertel',
                                                    'Allentown', 'ElmwoodBidwell', 'GeneseeMoselle', 'Kaisertown',
                                                    'UniversityHeights', 'KensingtonBailey', 'WestSide',
                                                    'MastenPark', 'UpperWestSide', 'LowerWestSide', 'BlackRock',
                                                    'FruitBelt', 'SchillerPark', 'FillmoreLeroy',
                                                    'SenecaCazenovia', 'Kenfield', 'Parkside', 'HopkinsTifft',
                                                    'CentralPark', 'SouthPark', 'Ellicott', 'MLKPark',
                                                    'GrantAmherst', 'DelavanGrider', 'HamlinPark', 'SenecaBabcock',
                                                    'FirstWard'])

In [7]:
df.nunique()

date    5841
nbh       35
day        7
hour      24
nb        14
dtype: int64

In [8]:
df.shape

(250728, 5)

In [9]:
df = df.dropna()

In [10]:
df.shape
df_= df
df_= df_.drop('date', axis=1)
df_.head()

,nbh,day,hour,nb
5,NorthPark,SUNDAY,0,10
14,Lovejoy,SATURDAY,19,8
27,PrattWillert,FRIDAY,16,8
28,Riverside,THURSDAY,0,7
45,BroadwayFillmore,WEDNESDAY,0,6


In [11]:
dummy_weekday = pd.get_dummies(df['day'])

dummy_weekday.columns = df['day'].unique()

df = pd.concat([df,dummy_weekday], axis=1)

dummy_hour= pd.get_dummies(df['hour'])

dummy_hour.columns = df['hour'].unique()

df = pd.concat([df,dummy_hour], axis=1)

dummy_nbh= pd.get_dummies(df['nbh'])

dummy_nbh.columns = df['nbh'].unique()

df = pd.concat([df,dummy_nbh], axis=1)

In [12]:
df= df.drop(['day', 'hour', 'nbh', 'date'], axis=1)

In [13]:
df_.head()

,nbh,day,hour,nb
5,NorthPark,SUNDAY,0,10
14,Lovejoy,SATURDAY,19,8
27,PrattWillert,FRIDAY,16,8
28,Riverside,THURSDAY,0,7
45,BroadwayFillmore,WEDNESDAY,0,6


In [14]:
from sklearn.model_selection import KFold # import KFold
from sklearn.metrics import accuracy_score #To calculate accuracy
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [15]:
kf = KFold(n_splits = 10)
def cross_validation_XG_classifier(model):
    accuracy = []
    for train_index, test_index in kf.split(df):
        #train part
        X_train = np.array(df.drop('nb', axis = 1))[train_index]
        y_train = np.array(df.nb)[train_index]
        #test part
        X_test = np.array(df.drop('nb', axis = 1))[test_index]
        y_test = np.array(df.nb)[test_index]

        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        y_pred = my_ceil(np.array(y_pred))

        accuracy.append(accuracy_score(y_pred, y_test))
    return np.average(accuracy)

In [16]:
# Converts floats to integers for classification
def my_ceil(predictions):
  for i in range(len(predictions)):

    if predictions[i]%1 <= +0.5:
      predictions[i] = int(predictions[i])
    else:
      predictions[i] = int(predictions[i]) + 1
  return predictions


def cross_validation_Lregressor(formula):
    accuracy = []
    for train_index, test_index in kf.split(df_):
    
        #train part
        train = df_.iloc[train_index]
        #test part
        test = df_.iloc[test_index]

        model_lr = formula
        result_lr = smf.ols(model_lr, data = train).fit()

        y_pred = np.array(result_lr.predict(test))

        y_pred = my_ceil(y_pred)

        accuracy.append(accuracy_score(y_pred, test.nb))
   
    return np.average(accuracy)

In [17]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [18]:
model_lr = 'nb ~ nbh + day + hour'
result_lr = smf.ols(model_lr, data = df_).fit()

In [19]:
cross_validation_Lregressor(model_lr)

0.8947325862047842

In [20]:
import xgboost

In [21]:
model_xg = xgboost.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.05,
                          alpha = 10,base_score= 0.4, booster='gbtree', n_estimators = 150)

In [22]:
cross_validation_XG_classifier(model_xg)

0.8947325862047842